In [ ]:
%tensorflow_version 1.14

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


**Problem 1** Looking back on the scratch


*   Had to initialize the weights.
*   Needed an epoch loop.
*   Train the model with batches.
*   Design the network.
*   Calculate the forward pass.
*   Calculate the loss using the loss function.
*   Update the weights (backpropagation).
*   Use trained model to infer validation data or test data.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

**Problem 2:**
Consider the correspondence between scratch and TensorFlow


In [71]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
#Load dataset
df = pd.read_csv("Iris.csv")
#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [79]:
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

In [80]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 3.4856, val_loss : 31.9409, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 1.6658, val_loss : 11.3622, acc : 0.000, val_acc : 0.000
Epoch 2, loss : 1.2576, val_loss : 8.1329, acc : 0.000, val_acc : 0.188
Epoch 3, loss : 0.8895, val_loss : 8.0925, acc : 0.000, val_acc : 0.250
Epoch 4, loss : 0.6792, val_loss : 6.1259, acc : 0.000, val_acc : 0.250
Epoch 5, loss : 0.4080, val_loss : 2.4915, acc : 0.000, val_acc : 0.375
Epoch 6, loss : 0.2190, val_loss : 2.1563, acc : 0.000, val_acc : 0.500
Epoch 7, loss : 0.1380, val_loss : 1.1806, acc : 0.250, val_acc : 0.625
Epoch 8, loss : 0.0667, val_loss : 0.7452, acc : 0.500, val_acc : 0.750
Epoch 9, loss : 0.0330, val_loss : 0.4585, acc : 1.000, val_acc : 0.812
Epoch 10, loss : 0.0150, val_loss : 0.2696, acc : 1.000, val_acc : 0.875
Epoch 11, loss : 0.0089, val_loss : 0.1959, acc : 1.000, val_acc : 0.938
Epoch 12, loss : 0.0056, val_loss : 0.1359, acc : 1.000, val_acc : 0.938
Epoch 13, loss : 0.0044, val_loss : 0.1107, acc : 1.000, va

*   Had to initialize the weights.
    ```
    init = tf.global_variables_initializer()
    ```

*   Needed an epoch loop.
    ```
    for epoch in range(num_epochs):
    ```

*   Train the model with batches.
    ```
    define the GetMiniBatch class
    ```

*   Design the network.
    ```
    define the example_net function
    ```

*   Calculate the forward pass.
    ```
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    ```

*   Calculate the loss using the loss function.
    ```
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logit(labels=Y, logits=logits))
    ```

*   Update the weights (backpropagation):
    ```
    loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
    ```

*   Use trained model to infer validation data or test data.
    ```
    val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
    ...
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    ```

**Problem 3** Create a model of Iris using all three types of objective variables

In [70]:
# Training on 3 classes
#Load dataset
df = pd.read_csv("Iris.csv")

#Condition extraction from data frame
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y[y == "Iris-setosa"] = 2

# Convert label to one-hot vector
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

In [ ]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

**Problem 4** Creating a model of House Prices

In [ ]:
# Load House Prices dataset
df_house = pd.read_csv('train.csv')

y = df_house[['SalePrice']]
X = df_house[['GrLivArea', 'YearBuilt']]
y = np.array(np.log1p(y))
X = np.array(np.log1p(X))

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
def regression_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

In [ ]:
# Hyperparameter settings
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = regression_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.square(Y - logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
#Index value calculation
mean_square_error = tf.reduce_mean(tf.square(Y - logits))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_mse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, mse = sess.run([loss_op, mean_square_error], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_mse += mse
        total_loss /= n_samples
        total_mse /= n_samples
        val_loss, val_mse = sess.run([loss_op, mean_square_error], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}, val_mse : {:.3f}".format(epoch, total_loss, val_loss, mse, val_mse))
    test_mse = sess.run(mean_square_error, feed_dict={X: X_test, Y: y_test})
    print("test_mse : {:.3f}".format(test_mse))

Epoch 0, loss : 205.8077, val_loss : 3.5418, mse : 3.646, val_mse : 3.542
Epoch 1, loss : 0.2607, val_loss : 1.4297, mse : 1.490, val_mse : 1.430
Epoch 2, loss : 0.0902, val_loss : 0.4609, mse : 0.673, val_mse : 0.461
Epoch 3, loss : 0.0298, val_loss : 0.1546, mse : 0.233, val_mse : 0.155
Epoch 4, loss : 0.0148, val_loss : 0.1194, mse : 0.075, val_mse : 0.119
Epoch 5, loss : 0.0154, val_loss : 0.2364, mse : 0.131, val_mse : 0.236
Epoch 6, loss : 0.0151, val_loss : 0.1928, mse : 0.096, val_mse : 0.193
Epoch 7, loss : 0.0121, val_loss : 0.1110, mse : 0.043, val_mse : 0.111
Epoch 8, loss : 0.0114, val_loss : 0.1044, mse : 0.038, val_mse : 0.104
Epoch 9, loss : 0.0117, val_loss : 0.1694, mse : 0.077, val_mse : 0.169
test_mse : 0.164


**Problem 5** Creating a MNIST model

In [66]:
from keras.datasets import mnist
(X, y), (X_test, y_test) = mnist.load_data()

In [67]:
# Smoothing
X = X.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [68]:
# Type conversion, normalization
X = X.astype(np.float)
X_test = X_test.astype(np.float)
X /= 255
X_test /= 255

In [69]:
# Convert label to one-hot vector
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:,np.newaxis])
y_test_one_hot = enc.transform(y_test[:,np.newaxis])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)

In [ ]:
def cnn(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([5,5,1,6])),
        'w2': tf.Variable(tf.random_normal([5,5,6,16])),
        'w3': tf.Variable(tf.random_normal([7*7*16, 120])),
        'w4': tf.Variable(tf.random_normal([120, 84])),
        'w5': tf.Variable(tf.random_normal([84, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([6])),
        'b2': tf.Variable(tf.random_normal([16])),
        'b3': tf.Variable(tf.random_normal([120])),
        'b4': tf.Variable(tf.random_normal([84])),
        'b5': tf.Variable(tf.random_normal([n_classes]))
    }

    x = tf.reshape(x, [-1, 28 , 28 , 1])

    conve_1 = tf.add(tf.nn.conv2d(x, weights['w1'], strides=[1,1,1,1], padding='SAME'), biases['b1'])
    conve_1 = tf.nn.relu(conve_1)

    pooli_1 = tf.nn.pool(conve_1, window_shape=[2,2], strides=[2,2], pooling_type='MAX', padding='VALID')

    conve_2 = tf.add(tf.nn.conv2d(pooli_1, weights['w2'], strides=[1,1,1,1], padding='SAME'), biases['b2'])
    conve_2 = tf.nn.relu(conve_2)

    pooli_2 = tf.nn.pool(conve_2, window_shape=[2,2], strides=[2,2], pooling_type='MAX', padding='VALID')

    x = tf.reshape(pooli_2, [-1, 7*7*16])
    
    layer_1 = tf.add(tf.matmul(x, weights['w3']), biases['b3'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w4']), biases['b4'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w5']) + biases['b5'] # tf.add and + are equivalent
    return layer_output

In [ ]:
  # Hyperparameter settings
learning_rate = 0.01
batch_size = 100
num_epochs = 50
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = cnn(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 6.4983, val_loss : 104.9089, acc : 0.920, val_acc : 0.888
Epoch 1, loss : 0.6259, val_loss : 48.2709, acc : 0.930, val_acc : 0.919
Epoch 2, loss : 0.3027, val_loss : 31.9755, acc : 0.940, val_acc : 0.935
Epoch 3, loss : 0.1812, val_loss : 22.8258, acc : 0.960, val_acc : 0.941
Epoch 4, loss : 0.1185, val_loss : 19.7515, acc : 0.940, val_acc : 0.941
Epoch 5, loss : 0.0849, val_loss : 15.6083, acc : 0.960, val_acc : 0.948
Epoch 6, loss : 0.0589, val_loss : 15.2744, acc : 0.930, val_acc : 0.945
Epoch 7, loss : 0.0443, val_loss : 12.4619, acc : 0.980, val_acc : 0.957
Epoch 8, loss : 0.0346, val_loss : 11.6242, acc : 0.960, val_acc : 0.953
Epoch 9, loss : 0.0288, val_loss : 8.7738, acc : 0.960, val_acc : 0.960
Epoch 10, loss : 0.0239, val_loss : 8.4185, acc : 0.970, val_acc : 0.958
Epoch 11, loss : 0.0190, val_loss : 7.7643, acc : 0.970, val_acc : 0.961
Epoch 12, loss : 0.0153, val_loss : 7.8986, acc : 0.980, val_acc : 0.954
Epoch 13, loss : 0.0146, val_loss : 8.0818, acc : 0